In [7]:
#!/usr/bin/env python2 or python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 12 2018

@author: jshermeyer

Random Forest Super-Resolution (RFSR)
Super resolves imagery by analyzing the relationships between adjacent LR pixels and the corresponding 
HR image pixel.  Uses a random forest regressor and a residual training schema.  Operates on the luminance
component of a YCbCr converted image.

"""

#imports
import sys
import os
import glob
import gdal
import numpy as np
import scipy
from scipy import misc
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import datetime
import cv2
from tqdm import tqdm_notebook as tqdm

#Various functions called throughout

#Crop out the center of an array.
def crop_center(array,cropx,cropy):
    y,x = array.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return array[starty:starty+cropy,startx:startx+cropx]

#Convert an RGB image to an YCbCr array
def Calc_YCbCr(RGB_Image):
    srcRaster=gdal.Open(RGB_Image)
    Blue = srcRaster.GetRasterBand(3).ReadAsArray()
    Green = srcRaster.GetRasterBand(2).ReadAsArray()
    Red = srcRaster.GetRasterBand(1).ReadAsArray()
    shape=Blue.shape
    Y=0+(0.2999 * Red)+(0.587 * Green)+(0.114 * Blue)
    CB=128-(0.168736 * Red)-(0.331264 * Green)+(0.5 * Blue)
    CR=128+(0.5 * Red)-(0.418688 * Green)-(0.081312 * Blue)
    outArray=np.asarray([CR,Y,CB])    
    return outArray

    
#Convert an RGB array to an YCbCr array    
def Calc_YCbCr_Array(RGB_Array):
    srcRaster=RGB_Array
    #print(srcRaster.shape)
    Blue = srcRaster[2,:,:]
    Green = srcRaster[1,:,:]
    Red = srcRaster[0,:,:]
    shape=Blue.shape
    #print(shape)
    Y=0+(0.2999 * Red)+(0.587 * Green)+(0.114 * Blue)
    CB=128-(0.168736 * Red)-(0.331264 * Green)+(0.5 * Blue)
    CR=128+(0.5 * Red)-(0.418688 * Green)-(0.081312 * Blue)
    outArray=np.asarray([CR,Y,CB])    
    return outArray

#Convert an YCbCr array to a RGB array  
def Calc_RGB_Array(YcBcR_Array):
    srcRaster=YcBcR_Array
    #print(srcRaster.shape)
    Cb = srcRaster[2,:,:]
    Y = srcRaster[1,:,:]
    Cr = srcRaster[0,:,:]
    shape=Y.shape
    #print(shape)
    R  = Y + (Cr - 128) *  1.40200
    G  = Y + (Cb - 128) * -0.34414 + (Cr - 128) * -0.71414
    B  = Y + (Cb - 128) *  1.77200
    outArray=np.asarray([R,G,B])    
    return outArray
     

#Upscale a lower resolution array to match an HR arrays shape using bicubic interpolation
def Upscale_Array(LR_Array,HR_Shape):
    RasHolder=[]
    for i in range(LR_Array.shape[0]):
        LR_band=LR_Array[i]
        LR_Resample=scipy.misc.imresize(LR_band,HR_Shape,interp='bicubic')
        RasHolder.append(LR_Resample)
    RasHolder=np.asarray(RasHolder)
    return RasHolder

#Shift an array around to create a 3D cube. 
#A shift_dimension of 1 takes into account the neighboring 1 pixels to the central pixel 
#https://i.stack.imgur.com/CWIHi.jpg (queen neighborhood)
#A shift_dimension of 2 takes into account the neighboring 2 pixels to the central pixel
#https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Ising_model_5x5_0.svg/2000px-Ising_model_5x5_0.svg.png
#(5x5 grid)
#shift_dimension of 3==7x7 grid, 4===9x9 grid
#The larger the grid the more memory intensive and more difficult training becomes
#Recommend a dimension of 2 or 3, but typically 2 is sufficient.

def Shift_Array(input_array,shift_dimension=2):
    shift_holder=[]
    current_shift_dimension_x=-shift_dimension
    current_shift_dimension_y=-shift_dimension    
    while current_shift_dimension_y<=shift_dimension:
        if current_shift_dimension_x < shift_dimension:
            Extent=np.roll(input_array,current_shift_dimension_y,axis=0)
            Extent=np.roll(Extent,current_shift_dimension_x,axis=1)
            shift_holder.append(Extent)           
            current_shift_dimension_x+=1
        elif current_shift_dimension_x == shift_dimension:
            Extent=np.roll(input_array,current_shift_dimension_y,axis=0)
            Extent=np.roll(Extent,current_shift_dimension_x,axis=1)
            shift_holder.append(Extent)
            current_shift_dimension_x=-shift_dimension
            current_shift_dimension_y+=1
            
    shift_holder=np.asarray(shift_holder).astype(int)
    return shift_holder

#Downscale an array by a set factor.
#If an image is 100x100 and the factor is 2, it will be downscaled to 50x50
#Set blur>0 to mimic the PSF of a camera as it moves further away from an object or the surface
#blur= the sigma of a gaussian blur you intend to mimic, we use 1.
#set inter_area==1 to use the more robust and accurate inter area decimation when degrading an image
#Otherwise, uses a bicubic decimation.
def Downscale_Array(HR_Array,factor=2,blur=0,inter_area=0):
    RasHolder=[]
    #print(HR_Array.shape)
    if blur > 0:
        HR_Array=np.swapaxes(HR_Array,0,2)
        blur_level=(factor/2)*blur
        #print(HR_Array.shape)
        HR_Array = cv2.GaussianBlur(HR_Array, (0, 0), blur_level, blur_level, 0)
        HR_Array=np.swapaxes(HR_Array,2,0)
        #print(HR_Array.shape)
    if inter_area > 0:
        del RasHolder
        #print(HR_Array.shape)
        HR_Array=np.swapaxes(HR_Array,0,2)
        #print(HR_Array.shape)
        #print(factor)
        x=(1/float(factor))
        #print(x)
        RasHolder=cv2.resize(HR_Array, (0,0), fx=x,fy=x, interpolation=cv2.INTER_AREA)
        RasHolder=np.swapaxes(RasHolder,0,2)
    else:
        for i in range(HR_Array.shape[0]):
            HR_band=HR_Array[i]
            HR_Shape=(int(HR_band.shape[0]/factor), int(HR_band.shape[1]/factor))
            HR_Resample=scipy.misc.imresize(HR_band,HR_Shape,interp='bicubic')
            RasHolder.append(HR_Resample)
        RasHolder=np.asarray(RasHolder)
    return RasHolder


#Output multi or single band geotiffs
def CreateMultiBandGeoTiff(Array, Name):
    driver=gdal.GetDriverByName('GTiff')
    DataSet = driver.Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], gdal.GDT_Float32)
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
    del DataSet
    return Name

def CreateSingleBandGeoTiff(Array, Name):
    driver=gdal.GetDriverByName('GTiff')
    DataSet = driver.Create(Name, Array.shape[1], Array.shape[0], 1, gdal.GDT_Float32)
    DataSet.GetRasterBand(1).WriteArray(Array)
    del DataSet
    return Name   
    


In [8]:
#Create training data as an input to train RFSR
#Index is the index of the luminance channel in a YCbCr converted image
#Set =1 if using my functions for conversion, 0 for most others.
def Create_SR_TrainingData(LR_Array,HR_Array,shift_dimension=2,index=1):
    shift_window=((shift_dimension * 2) + 1) ** 2

    #Upscale the raster to match HR_Raster
    size1=HR_Array.shape[1]
    size2=HR_Array.shape[2]
    Bicube_LR=Upscale_Array(LR_Array,(size1,size2))
    
    HR_Shape=(LR_Array.shape[0],size1,size2)

    #Convert to YCbCr
    Bicube_LR=(Calc_YCbCr_Array(Bicube_LR)[index,:,:])
    HR_Array=(Calc_YCbCr_Array(HR_Array)[index,:,:]) 
    
    
    #If imagery is irregularly shaped, you may want to uncomment this
    #cropx=(HR_Shape[2])
    #cropy=(HR_Shape[1])
    #HR_Array=crop_center(HR_Array,cropx,cropy)
    
    #Optionally Pad
    Bicube_LR = np.pad(Bicube_LR, pad_width=shift_dimension, mode='constant', constant_values=0)
    HR_Array = np.pad(HR_Array, pad_width=shift_dimension, mode='constant', constant_values=0)

    #Reshape the HR to 3D
    HR_Array=HR_Array.reshape(1,HR_Shape[1]+(shift_dimension*2),HR_Shape[2]+(shift_dimension*2))

    #Create patches
    BC_LR_Shift=Shift_Array(Bicube_LR,shift_dimension=shift_dimension)
    
    #Subtract the Bicube_LR upscaled image
    BC_LR_Shift=BC_LR_Shift-Bicube_LR
    HR_Array=HR_Array-Bicube_LR    
    
    #Append the Arrays to ensure they are consistently located in 2d space before converting to 1d
    Append_Array=np.append(BC_LR_Shift,HR_Array, axis=0)
    nfeatures, nx, ny = Append_Array.shape
    Append_Array=Append_Array.reshape(nfeatures,(nx*ny))
    Append_Array=np.swapaxes(Append_Array,0,1)
    Train=Append_Array[:,0:shift_window]
    Target=Append_Array[:,shift_window]
    
    return Train, Target

In [8]:
## After importing all functions, start here

#Path to your high res imagery.
HR_Path="/HR_Images"
os.chdir(HR_Path)

#The scale of enhancement
ScalingFactor=2

#The sigma of your gaussian blur,if blur = 0, no blurring happens, if > 0 sigma=blur
blur=1

## If -inter_area=1, cv2 interarea, otherwise bicubic decimation
inter_area=1

#Change this to whatever your extension is.
HR=glob.glob('*.tif')

#shift_dimension- see description above in the functions, I'd use 2, and perhaps 3 if you have time to kill.
shift_dimension=2  
    

In [ ]:
#Create a training and testing dataset for scoring
HR.sort()
HR_Rasters=[]
for raster in HR:
    raster=gdal.Open(raster).ReadAsArray()
    HR_Rasters.append(raster)
    
LR_Rasters=[]
for raster in HR_Rasters:
    LR_raster=Downscale_Array(raster,factor=ScalingFactor,blur=blur,inter_area=inter_area)
    LR_Rasters.append(LR_raster)

Train_TrainSet=[]
Train_TestSet=[]
Target_TrainSet=[]
Target_TestSet=[]
count=1
t1=datetime.datetime.now()
print(datetime.datetime.now())
for LR, HR in (zip(LR_Rasters,HR_Rasters)):
    train, target = Create_SR_TrainingData(LR,HR,shift_dimension=shift_dimension)
    Train_TrainSet_temp, Train_TestSet_temp, Target_TrainSet_temp, Target_TestSet_temp = train_test_split(train, target, test_size=0.1,train_size=0.1)
    
    Train_TrainSet.append(Train_TrainSet_temp)
    Train_TestSet.append(Train_TestSet_temp)
    Target_TrainSet.append(Target_TrainSet_temp)
    Target_TestSet.append(Target_TestSet_temp)
    
    Train_TrainSet=[np.concatenate(Train_TrainSet,axis=0)]
    Train_TestSet=[np.concatenate(Train_TestSet,axis=0)]  
    Target_TrainSet=[np.concatenate(Target_TrainSet,axis=0)]
    Target_TestSet=[np.concatenate(Target_TestSet,axis=0)]
    if count % 100 == 0:
        print(count)
    count+=1

print(datetime.datetime.now())
t2=datetime.datetime.now()
print(t2-t1)
    

In [10]:
#Check to ensure the first dimension of the shapes match.
print(Train_TrainSet[0].shape)
print(Target_TrainSet[0].shape)

(20330310, 25)
(20330310,)


In [ ]:
#Sample the training and testing set at 10% rate.  May want to increase this depending on your data.
t1=datetime.datetime.now()
print(datetime.datetime.now())
#Default settings listed, adjust as you see fit.
rf = RandomForestRegressor(n_estimators=100, max_depth=12,min_samples_split=200, n_jobs=-1,verbose=1,oob_score=True)
print(rf)
OutputModel=rf.fit(Train_TrainSet[0],Target_TrainSet[0])
print(datetime.datetime.now())
t2=datetime.datetime.now()
print(t2-t1)
mse=mean_squared_error( Target_TestSet[0],rf.predict(Train_TestSet[0]))
#Training and testing scores, these will be close to what you will expect to see for a larger dataset
print("MSE:",mse)
print("PSNR:",20 * np.log10(255 / np.sqrt((mse)))

In [5]:
#Save your model somewhere
joblib.dump(rf, '/models/rfmodel1.pkl')

In [ ]:
#Inference code
#shift_dimension again, must be identical to how you trained your model
shift_dimension=2
#Scaling factor
SF=2
#Load a model
rf = joblib.load("/RF_Models/RF_2x_1Sigma_30cm_10percent_5x5PatchSize_Pad_100Depth_IA.pkl")
#These should be your low resolution images.  This code does not degrade then reupsample imagery.
LR_Path="/images_to_be_enhanced"
#output directory
output_dir="/output_path"
os.chdir(LR_Path)
#Again change this to your extension.
LR=glob.glob('*.tif')


LR.sort()
LR_Rasters=[]
driver = gdal.GetDriverByName("GTiff")
for image in tqdm(LR):
    raster=gdal.Open(image)
    geo = raster.GetGeoTransform()
    pixW=float(geo[1])/SF
    pixH=float(geo[5])/SF
    geo=[geo[0],pixW,geo[2],geo[3],geo[4],pixH]
    proj = raster.GetProjection()
    raster=raster.ReadAsArray()
    #print(raster.shape)
    size1=raster.shape[1]*SF
    size2=raster.shape[2]*SF
    raster=Upscale_Array(raster,(size1,size2))
    
    #Convert to YCbCr
    YCbCr=(Calc_YCbCr_Array(raster))
    Y=YCbCr[1,:,:]
    Cr=YCbCr[0,:,:]
    Cb=YCbCr[2,:,:]
    
    # Pad
    Y = np.pad(Y, pad_width=shift_dimension, mode='constant', constant_values=0)
    Cr = np.pad(Cr, pad_width=shift_dimension, mode='constant', constant_values=0)
    Cb = np.pad(Cb, pad_width=shift_dimension, mode='constant', constant_values=0)
    
    #print(Y.shape,Cb.shape,Cr.shape)

    #Create patches
    Y_Shift=Shift_Array(Y,shift_dimension=shift_dimension)
    
    #Subtract the Bicube_LR upscaled image
    Y_Shift=Y_Shift-Y 
    
    #Create model ready input
    nfeatures, nx, ny = Y_Shift.shape
    Input=Y_Shift.reshape(nfeatures,(nx*ny))
    Input=np.swapaxes(Input,0,1)
    
    #Infer
    Output=rf.predict(Input)
    Output=Output.reshape(Output.shape[0]//ny,-1)
    #print(Output.shape)
    Output=Output+Y
    Stack=np.array([Cr,Output,Cb])
    Stack=Stack[:, shift_dimension:-shift_dimension, shift_dimension:-shift_dimension]
    Stack=Calc_RGB_Array(Stack)
    
    #Save
    out=output_dir+str(image)
    #print(out)
    DataSet = driver.Create(out, Stack.shape[2], Stack.shape[1], Stack.shape[0], gdal.GDT_Byte)
    for i, B in enumerate(Stack, 1):
        DataSet.GetRasterBand(i).WriteArray( B )
    DataSet.SetProjection(proj)
    DataSet.SetGeoTransform(geo)
    #DataSet.SetNoDataValue(0)
    del DataSet

In [ ]:
#Code for simply testing your performance across multiple scaling factors for a different or whole dataset.

from skimage.measure import compare_ssim

shift_dimension=2
SFs=[2,4,8]
rfs=[]
#Load models.
rf = joblib.load("/RF_Models/RF_2x_1Sigma_120cm_10percent_5x5PatchSize_Pad_100Depth_IA.pkl")
rfs.append(rf)
rf = joblib.load("/RF_Models/RF_4x_1Sigma_120cm_10percent_5x5PatchSize_Pad_100Depth_IA.pkl")
rfs.append(rf)
rf = joblib.load("/RF_Models/RF_8x_1Sigma_120cm_10percent_5x5PatchSize_Pad_100Depth_IA.pkl")
rfs.append(rf)
#This code DOES degrade and then upscale.  If you want to test performance for 30cm SR output feed in 30cm data
LR_Path="/ImageryPath/"
os.chdir(LR_Path)
#extension.
LR=glob.glob('*.tif')
LR.sort()

for SF,rf in zip(SFs,rfs):
    LR_Rasters=[]
    mse=[]
    ssim=[]
    driver = gdal.GetDriverByName("GTiff")
    count=0
    for image in tqdm(LR):
        count+=1
        raster=gdal.Open(image)
        raster=raster.ReadAsArray()
        YCbCr=(Calc_YCbCr_Array(raster))
        HR_Y=YCbCr[1,:,:]
        raster=Downscale_Array(raster,factor=SF,blur=1,inter_area=1)
        raster=Upscale_Array(raster,HR_Y.shape)


        #Convert to YCbCr
        YCbCr=(Calc_YCbCr_Array(raster))
        Y=YCbCr[1,:,:]

        # Pad
        Y = np.pad(Y, pad_width=shift_dimension, mode='constant', constant_values=0)

        #Create patches
        Y_Shift=Shift_Array(Y,shift_dimension=shift_dimension)

        #Subtract the Bicube_LR upscaled image
        Y_Shift=Y_Shift-Y 

        #Create model ready input
        nfeatures, nx, ny = Y_Shift.shape
        Input=Y_Shift.reshape(nfeatures,(nx*ny))
        Input=np.swapaxes(Input,0,1)

        #Infer
        Output=rf.predict(Input)
        Output=Output.reshape(Output.shape[0]//ny,-1)
        Output=Output+Y
        Output=Output[shift_dimension:-shift_dimension, shift_dimension:-shift_dimension]


        #Test
        mse.append(mean_squared_error(HR_Y,Output))
        ssim.append(compare_ssim(HR_Y, Output, data_range=Output.max() - Output.min()))
        #if count % 50 == 0:
            #print("MSE:",np.mean(mse))
            #print("SSIM:",np.nanmean(ssim))


    print("SF:", SF)
    print("MSE:",np.mean(mse))
    print("PSNR:",20 * np.log10(255 / np.sqrt(np.mean(mse))))
    print("SSIM:",np.nanmean(ssim))